# Claim-based Question-Answer Hallucination Detection

Import necessary packages.

In [1]:
import time
from uqlm.longform.black_box import ClaimQAScorer
from uqlm import BlackBoxUQ

#### Load LLM and device

In [2]:
# from langchain_google_vertexai import ChatVertexAI
# llm = ChatVertexAI(model="gemini-1.5-flash")

from dotenv import load_dotenv, find_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv(find_dotenv())
llm = AzureChatOpenAI(
    deployment_name="gpt-4o-mini",
    openai_api_type="azure",
    openai_api_version="2024-02-15-preview",
    temperature=1,  # User to set temperature
)

In [3]:
import torch

# Set the torch device
if torch.cuda.is_available():  # NVIDIA GPU
    device = torch.device("cuda")
elif torch.backends.mps.is_available():  # macOS
    device = torch.device("mps")
else:
    device = torch.device("cpu")  # CPU
print(f"Using {device.type} device")

Using mps device


#### Setup Prompts and Black Box Scorer

In [4]:
prompts = [
    "write a paragraph about Paul McCartney"
    # "write a paragraph about John Lennon"
]

In [5]:
bb_scorer = BlackBoxUQ(
    llm=llm,
    max_calls_per_min=500,  # set value to avoid rate limit error
    device=device,
    scorers=["exact_match"],
)

#### Claim-QA class

In [6]:
start_time = time.time()
claim_qa = ClaimQAScorer(llm=llm, black_box_scorers=["exact_match"], response_template="atomic", max_calls_per_min=500)
result = await claim_qa.generate_and_score(prompts=prompts)
print(f"Computation time: {time.time() - start_time} seconds")

Claim sets:  [['Paul McCartney is a legendary British musician.', 'Paul McCartney is a songwriter.', 'Paul McCartney is a producer.', 'Paul McCartney is best known as a member of The Beatles.', 'The Beatles is one of the most influential bands in the history of music.', 'Paul McCartney was born on June 18, 1942.', 'Paul McCartney was born in Liverpool, England.', 'Paul McCartney showcased his musical talent from a young age.', "Paul McCartney played a pivotal role in The Beatles' unparalleled success during the 1960s.", 'Paul McCartney crafted timeless classics such as "Hey Jude."', 'Paul McCartney crafted timeless classics such as "Yesterday."', 'Paul McCartney crafted timeless classics such as "Let It Be."', "Paul McCartney embarked on a prolific solo career following The Beatles' breakup in 1970.", 'Paul McCartney formed the band Wings.', 'Paul McCartney released numerous albums.', "Paul McCartney's albums further solidified his status as a musical icon.", "Paul McCartney's career s